In [1]:
import os
import pathlib

import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.losses import CategoricalCrossentropy

In [2]:
CLASSES = ['rock', 'paper', 'scissors']
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
def load_image_and_label(image_path, target_size=(32, 32)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.convert_image_dtype(image, np.float32)
    image = tf.image.resize(image, target_size)

    label = tf.strings.split(image_path, os.path.sep)[-2]
    label = (label == CLASSES)  # One-hot encode.
    label = tf.dtypes.cast(label, tf.float32)

    return image, label

In [4]:
def build_network():
    input_layer = Input(shape=(32, 32, 1))
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same',
               strides=(1, 1))(input_layer)
    x = ReLU()(x)
    x = Dropout(rate=0.5)(x)

    x = Flatten()(x)
    x = Dense(units=3)(x)
    output = Softmax()(x)

    model = Model(inputs=input_layer, outputs=output)
    return model


In [5]:
def prepare_dataset(dataset_path,
                    buffer_size,
                    batch_size,
                    shuffle=True):
    dataset = (tf.data.Dataset
               .from_tensor_slices(dataset_path)
               .map(load_image_and_label,
                    num_parallel_calls=AUTOTUNE))

    if shuffle:
        dataset.shuffle(buffer_size=buffer_size)

    dataset = (dataset
               .batch(batch_size=batch_size)
               .prefetch(buffer_size=buffer_size))

    return dataset

In [6]:
file_patten = ('C:/Users/hp/Desktop/multi-class classifier to play rock paper scissors/data/rps-cv-images/*/*.png')
file_pattern = str(file_patten)
dataset_paths = [*glob.glob(file_pattern)]

In [7]:
train_paths, test_paths = train_test_split(dataset_paths,
                                           test_size=0.2,
                                           random_state=999)
train_paths, val_paths = train_test_split(train_paths,
                                          test_size=0.2,
                                          random_state=999)

In [8]:
BATCH_SIZE = 1024
BUFFER_SIZE = 1024

train_dataset = prepare_dataset(train_paths,
                                buffer_size=BUFFER_SIZE,
                                batch_size=BATCH_SIZE)
validation_dataset = prepare_dataset(val_paths,
                                     buffer_size=BUFFER_SIZE,
                                     batch_size=BATCH_SIZE,
                                     shuffle=False)
test_dataset = prepare_dataset(test_paths,
                               buffer_size=BUFFER_SIZE,
                               batch_size=BATCH_SIZE,
                               shuffle=False)

In [9]:
model = build_network()
model.compile(loss=CategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
EPOCHS = 250
model.fit(train_dataset,
          validation_data=validation_dataset,
          epochs=EPOCHS)

Epoch 1/250


C:\Users\hp\anaconda3\lib\site-packages\keras\backend.py:5531: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2/2 [==============================] - 3s 846ms/step - loss: 1.3612 - accuracy: 0.3257 - val_loss: 1.2571 - val_accuracy: 0.3200
Epoch 2/250
2/2 [==============================] - 2s 620ms/step - loss: 1.1999 - accuracy: 0.3593 - val_loss: 1.4081 - val_accuracy: 0.3771
Epoch 3/250
2/2 [==============================] - 2s 599ms/step - loss: 1.4987 - accuracy: 0.3214 - val_loss: 1.1978 - val_accuracy: 0.3857
Epoch 4/250
2/2 [==============================] - 2s 579ms/step - loss: 1.2242 - accuracy: 0.3643 - val_loss: 1.1264 - val_accuracy: 0.3743
Epoch 5/250
2/2 [==============================] - 2s 766ms/step - loss: 1.1230 - accuracy: 0.3686 - val_loss: 1.2752 - val_accuracy: 0.3029
Epoch 6/250
2/2 [==============================] - 2s 635ms/step - loss: 1.2049 - accuracy: 0.3357 - val_loss: 1.1557 - val_accuracy: 0.3086
Epoch 7/250
2/2 [==============================] - 2s 620ms/step - loss: 1.0939 - accuracy: 0.3729 - val_loss: 1.0529 - val_accuracy: 0.5000
Epoch 8/250
2/2 [========

Epoch 59/250
2/2 [==============================] - 2s 601ms/step - loss: 0.4170 - accuracy: 0.8864 - val_loss: 0.4945 - val_accuracy: 0.8657
Epoch 60/250
2/2 [==============================] - 2s 557ms/step - loss: 0.4052 - accuracy: 0.8864 - val_loss: 0.4861 - val_accuracy: 0.8629
Epoch 61/250
2/2 [==============================] - 2s 566ms/step - loss: 0.4031 - accuracy: 0.8971 - val_loss: 0.4823 - val_accuracy: 0.8657
Epoch 62/250
2/2 [==============================] - 2s 575ms/step - loss: 0.3995 - accuracy: 0.8836 - val_loss: 0.4727 - val_accuracy: 0.8571
Epoch 63/250
2/2 [==============================] - 2s 579ms/step - loss: 0.3905 - accuracy: 0.8986 - val_loss: 0.4673 - val_accuracy: 0.8657
Epoch 64/250
2/2 [==============================] - 2s 564ms/step - loss: 0.3859 - accuracy: 0.8936 - val_loss: 0.4595 - val_accuracy: 0.8600
Epoch 65/250
2/2 [==============================] - 2s 590ms/step - loss: 0.3766 - accuracy: 0.8986 - val_loss: 0.4539 - val_accuracy: 0.8657
Epoch 

2/2 [==============================] - 2s 590ms/step - loss: 0.1833 - accuracy: 0.9621 - val_loss: 0.2949 - val_accuracy: 0.8971
Epoch 117/250
2/2 [==============================] - 2s 605ms/step - loss: 0.1837 - accuracy: 0.9629 - val_loss: 0.2928 - val_accuracy: 0.9029
Epoch 118/250
2/2 [==============================] - 2s 576ms/step - loss: 0.1783 - accuracy: 0.9664 - val_loss: 0.2934 - val_accuracy: 0.8971
Epoch 119/250
2/2 [==============================] - 2s 579ms/step - loss: 0.1792 - accuracy: 0.9657 - val_loss: 0.2905 - val_accuracy: 0.9029
Epoch 120/250
2/2 [==============================] - 2s 583ms/step - loss: 0.1763 - accuracy: 0.9607 - val_loss: 0.2906 - val_accuracy: 0.9000
Epoch 121/250
2/2 [==============================] - 2s 585ms/step - loss: 0.1724 - accuracy: 0.9664 - val_loss: 0.2875 - val_accuracy: 0.9029
Epoch 122/250
2/2 [==============================] - 2s 656ms/step - loss: 0.1663 - accuracy: 0.9707 - val_loss: 0.2860 - val_accuracy: 0.9029
Epoch 123/250

2/2 [==============================] - 2s 590ms/step - loss: 0.1145 - accuracy: 0.9800 - val_loss: 0.2500 - val_accuracy: 0.9114
Epoch 174/250
2/2 [==============================] - 2s 584ms/step - loss: 0.1074 - accuracy: 0.9800 - val_loss: 0.2475 - val_accuracy: 0.9171
Epoch 175/250
2/2 [==============================] - 2s 558ms/step - loss: 0.1124 - accuracy: 0.9814 - val_loss: 0.2465 - val_accuracy: 0.9143
Epoch 176/250
2/2 [==============================] - 2s 552ms/step - loss: 0.1067 - accuracy: 0.9807 - val_loss: 0.2465 - val_accuracy: 0.9171
Epoch 177/250
2/2 [==============================] - 2s 589ms/step - loss: 0.1059 - accuracy: 0.9821 - val_loss: 0.2479 - val_accuracy: 0.9143
Epoch 178/250
2/2 [==============================] - 2s 569ms/step - loss: 0.1044 - accuracy: 0.9821 - val_loss: 0.2463 - val_accuracy: 0.9200
Epoch 179/250
2/2 [==============================] - 2s 601ms/step - loss: 0.1041 - accuracy: 0.9836 - val_loss: 0.2442 - val_accuracy: 0.9143
Epoch 180/250

2/2 [==============================] - 2s 567ms/step - loss: 0.0725 - accuracy: 0.9879 - val_loss: 0.2307 - val_accuracy: 0.9114
Epoch 231/250
2/2 [==============================] - 2s 577ms/step - loss: 0.0722 - accuracy: 0.9886 - val_loss: 0.2278 - val_accuracy: 0.9143
Epoch 232/250
2/2 [==============================] - 2s 560ms/step - loss: 0.0750 - accuracy: 0.9871 - val_loss: 0.2277 - val_accuracy: 0.9200
Epoch 233/250
2/2 [==============================] - 2s 552ms/step - loss: 0.0725 - accuracy: 0.9893 - val_loss: 0.2246 - val_accuracy: 0.9114
Epoch 234/250
2/2 [==============================] - 2s 615ms/step - loss: 0.0701 - accuracy: 0.9914 - val_loss: 0.2246 - val_accuracy: 0.9114
Epoch 235/250
2/2 [==============================] - 2s 549ms/step - loss: 0.0694 - accuracy: 0.9900 - val_loss: 0.2264 - val_accuracy: 0.9229
Epoch 236/250
2/2 [==============================] - 2s 615ms/step - loss: 0.0698 - accuracy: 0.9864 - val_loss: 0.2242 - val_accuracy: 0.9143
Epoch 237/250

In [11]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Loss: {test_loss}, accuracy: {test_accuracy}')

1/1 [==============================] - 1s 786ms/step - loss: 0.2239 - accuracy: 0.9292
Loss: 0.2239052653312683, accuracy: 0.9292237162590027
